In [1]:
! pip install pandas 
! pip install requests

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.0 kB 660.6 kB/s eta 0:00:01
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -- ------------------------------------- 0.6/11.6 MB 12.4 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.6 MB 48.0 MB/s eta 0:00:01
   ----------------------------------- ---- 10.2/11.6 MB 72.3 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 81.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------- ----------------------------- 4.3/15.8 MB 139.7 MB/s eta 0:00:01
   ------------- -------------------------- 5.2/15.8 MB 111.3 MB/s eta 0:00:01
   ------------- -------------------------- 5.2/15.8 MB 111.3 MB/s eta 0:00:01
   ------------- -------------------------- 5.2/15.8 MB 111.3 MB/s eta 0:00:0

In [2]:
# Extrae todos los datos de trabajos por categoría de GetOnBrd y los guarda en un archivo CSV

import requests
import csv

# Obtener todas las categorías
categories_url = "https://www.getonbrd.com/api/v0/categories?per_page=100&page=1"
categories_response = requests.get(categories_url)

if categories_response.status_code == 200:
    categories_data = categories_response.json()
else:
    print(f"Error: {categories_response.status_code}")
    categories_data = None

if categories_data:
    all_jobs = []
    for category in categories_data['data']:
        category_id = category['id']
        page = 1
        while True:
            jobs_url = f"https://www.getonbrd.com/api/v0/categories/{category_id}/jobs?per_page=100&page={page}&expand=[%22company%22]"
            jobs_response = requests.get(jobs_url)
            
            if jobs_response.status_code == 200:
                jobs_data = jobs_response.json()
            else:
                print(f"Error: {jobs_response.status_code}")
                jobs_data = None

            if jobs_data and "data" in jobs_data and jobs_data["data"]:
                for job in jobs_data["data"]:
                    filtered_job = {
                        "category": job['attributes']['category_name'],
                        "seniority": job['attributes']['seniority'],
                        "min_salary": job['attributes']['min_salary'],
                        "max_salary": job['attributes']['max_salary'],
                        "id": job['id'],
                        "modality": job['attributes']['modality'],
                        "remote": job['attributes']['remote'],
                        "remote_modality": job['attributes']['remote_modality'],
                        "remote_zone": job['attributes']['remote_zone'],
                        "published_at": job['attributes']['published_at'],
                        "company": {
                            "name": job['attributes']['company']['data']['attributes']['name'],
                            "country": job['attributes']['company']['data']['attributes']['country'],
                            "id": job['attributes']['company']['data']['id'],
                            "description": job['attributes']['company']['data']['attributes']['description']
                        }
                    }
                    all_jobs.append(filtered_job)
                page += 1
            else:
                break

    # Guardar trabajos en un archivo CSV
    with open('../data/all_jobs_data.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Escribir los encabezados
        writer.writerow(["category", "seniority", "min_salary", "max_salary", "id", "modality", "remote", "remote_modality", "remote_zone", "published_at", "company_name", "company_country", "company_id", "company_description"])

        for job in all_jobs:
            writer.writerow([
                job["category"],
                job["seniority"]["data"]["id"],
                job["min_salary"],
                job["max_salary"],
                job["id"],
                job["modality"]["data"]["id"],
                job["remote"],
                job["remote_modality"],
                job["remote_zone"],
                job["published_at"],
                job["company"]["name"],
                job["company"]["country"],
                job["company"]["id"],
                job["company"]["description"]
            ])

    print("Todos los trabajos se han guardado en 'all_jobs_data.csv' correctamente.")
else:
    print("No se pudieron obtener las categorías.")


Todos los trabajos se han guardado en 'all_jobs_data.csv' correctamente.
